In [ ]:
# default_exp masks

In [ ]:
#export
from speechsep.imports import *
from speechsep.plot import *
from speechsep.utils import *
from speechsep.base import *
from speechsep.core import *

#hide
fn_list = Path("../data/SmallDataset/esc50_sample/").glob(r"*/*")
audio_list = [AudioBase(*load(fn)) for fn in fn_list]

# Sound Separation Masks
> All AudioMasking techniques that I know of.

In [ ]:
#export
def safe_div(x,y):
    return np.divide(x,y, out=np.zeros_like(x.data), where=y!=0)

In [ ]:
#export
class MaskBase():
    _show_args={}
    def __init__(self, data): store_attr(self, 'data')
    @property
    def shape(self):
        return self.data.shape
    @classmethod
    def create(cls, specs, mix_spec):
        return [cls.generate(spec, mix_spec) for spec in specs]
    def __mul__(self, spec):
        raise NotImplementedError('This function needs to be implemented before use')
    def __rmul__(self, spec):
        return self*spec
    def generate(self, spec, mix_spec):
        raise NotImplementedError('This function needs to be implemented before use')
    @delegates(setup_graph)
    def show(self, ctx=None, **kwargs): return show_mask(self, ctx=ctx, **merge(self._show_args, kwargs))


In [ ]:
MaskBase._tensor_cls = TensorMask
@ToTensor
def encodes(self, o:MaskBase):  return o._tensor_cls(mask2tensor(o))
def mask2tensor(mask:MaskBase):  return TensorMask(mask.data)

# Mask Types

## Binary Mask

In [ ]:
#export
class MaskBinary(MaskBase):
    def __init__(self, data, threshold=1):
        store_attr(self, 'data')
    def __mul__(self, spec):
        if isinstance(spec, torch.Tensor): return spec*self.data
        return SpecBase(spec.data*self.data, spec.sr, spec.fn)
 
    @classmethod
    def generate(cls, spec, mix_spec):
        Binary = (safe_div(abs(spec.data),abs(mix_spec.data)) >= 1)*1
        return cls(Binary)

## Complex Ideal Ratio Mask

In [ ]:
#export
class MaskcIRM(MaskBase):
    def __mul__(self, spec):
        if isinstance(spec, SpecBase):
            return SpecBase(spec.data*self.data, spec.sr)
        elif self.data.requires_grad == True or spec.requires_grad == True:
            return complex_mult(self.data, spec)
        return real2complex(spec)*real2complex(self.data)

    @classmethod
    def generate(cls, spec, mix_spec):
        cIRM = safe_div(spec.data, mix_spec.data)
        return cls(cIRM)

# Maskify Transform

In [ ]:
#export
class Maskify(TupleTransform):
    as_item_force=True
    def __init__(self, MaskType=MaskcIRM, Aud2Spec=Spectify()):
        store_attr(self, "MaskType, Aud2Spec")
    def encodes(self, audioList)->None:
        specList = [self.Aud2Spec(a) for a in audioList]
        joined = AudioBase(join_audios(audioList), audioList[0].sr)
        mix_spec = self.Aud2Spec(joined)
        maskList = self.MaskType.create(specList, mix_spec)
        return mix_spec, maskList
    def decodes(self, spec_mask)->None:
        mix_spec, maskList = spec_mask
        return [self.Aud2Spec.decode(spec*m) for m in maskList]